<a href="https://colab.research.google.com/github/kglobe/tw_stock/blob/master/%E4%B8%8A%E6%AB%83%E6%94%B6%E7%9B%A4%E5%A4%A7%E6%96%BC5MA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import requests
import json
import pandas as pd
import datetime
import time
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import re 
from google.colab import output
import xlrd

def getConvertData(convertType,input):
  try:
      getValue = eval(convertType)(input)
  except Exception:
      getValue = str(input)

  return getValue

def getCompLastMonth(row_array):
  try:
    if float(row_array[3]) == 0:
        return 0
    else:
        return (row_array[4]-row_array[3])*100/row_array[3]
  except Exception as e:
    print('getCompLastMonth Exception:',e)
    return 0

def getCompLastYear(row_array):
  try:
    if float(row_array[5]) == 0:
        return 0
    else:
        return (row_array[4]-row_array[5])*100/row_array[5]
  except Exception as e:
    print('getCompLastYear Exception:',e)
    return 0

def monthly_report_all(year, month):
    
  if month<10:
      url = 'https://www.tpex.org.tw/storage/statistic/sales_revenue/O_'+str(year)+'0'+str(month)+'.xls'
  else:
      url = 'https://www.tpex.org.tw/storage/statistic/sales_revenue/O_'+str(year)+str(month)+'.xls'
  
  # 偽瀏覽器
  # headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
  
  try:
      s = requests.Session()
      s.config = {'keep_alive': False}
      headers = {
          'Content-Type': 'application/x-www-form-urlencoded',
          'Connection': 'close',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
      }
      r = requests.get(url, headers=headers,  timeout=5, allow_redirects=True)
      r.encoding = 'BIG5'
      with open('O_'+str(year)+str(month)+'.xls', 'wb') as f:
          f.write(r.content)
      
  except Exception as e:
      print(str(year)+str(month)+' 抓資料有問題：'+str(e)+' , '+url)
  finally:
      s.close()
      # print('Session closed!')
      # return

  workbook = xlrd.open_workbook('O_'+str(year)+str(month)+'.xls')
  sheets = workbook.sheets()
  columns = ['年度月份','公司代號','公司名稱','上月營收','本月營收','去年同月營收','單月月增率','單月年增率','累計營收','去年累計營收','累積年增率']
  all_row = []
  for sheet in sheets:
      num_rows = sheet.nrows
      curr_row = -1
      while curr_row < num_rows:
          row_array = []
          curr_row += 1
          try:
              row = sheet.row(curr_row)
          except Exception :
              break
          cell_type = sheet.cell_type(curr_row, 0)
          if cell_type == 1:
              cell_value = sheet.cell_value(curr_row, 0)
              if cell_value[:4].isnumeric():
                  
                  row_array.append(str(year)+str(month).zfill(2))
                  row_array.append(getConvertData('str',cell_value[:4]))
                  row_array.append(getConvertData('str',cell_value[5:]))
                  
                  cell_value = sheet.cell_value(curr_row, 2)
                  if cell_value == 'NA' or cell_value == '':
                      continue

                  row_array.append(getConvertData('float',cell_value))

                  cell_value = sheet.cell_value(curr_row, 3)
                  row_array.append(getConvertData('float',cell_value))

                  cell_value = sheet.cell_value(curr_row, 5)
                  row_array.append(getConvertData('float',cell_value))

                  row_array.append(round(getCompLastMonth(row_array),6))
                  row_array.append(round(getCompLastYear(row_array),6))

                  cell_value = sheet.cell_value(curr_row, 4)
                  row_array.append(getConvertData('float',cell_value))

                  cell_value = sheet.cell_value(curr_row, 6)
                  row_array.append(getConvertData('float',cell_value))

                  cell_value = sheet.cell_value(curr_row, 8)
                  if cell_value == 'NA' or cell_value == '':
                      row_array.append(None)
                  else:
                      row_array.append(round(getConvertData('float',cell_value),6))
          
          if len(row_array) != 0:
              all_row.append(row_array)

  rtl_df = pd.DataFrame(np.array(all_row),columns=columns)
  return rtl_df

def getPayload(stockCode, year, month):
  payload = {
      'encodeURIComponent': 1,
      'step': 1,
      'firstin': 1,
      'off': 1,
      'queryName': stockCode,
      'inpuType': stockCode,
      'TYPEK': 'all',
      'isnew': 'false',
      'co_id': stockCode,
      'year': year,
      'month': '{:02d}'.format(month)}
  return payload
 
#取得當月營收
def monthly_report(stockCode, year, month):
    time.sleep(5)
    td_array = None
    # 假如是西元，轉成民國
    if year > 1990:
        year -= 1911
    #選歷史資料後按搜尋
    payload = getPayload(stockCode, year, month)
 
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    
    try:
        s = requests.Session()
        s.config = {'keep_alive': False}
        r = requests.post('https://mops.twse.com.tw/mops/web/t05st10_ifrs', data=payload, headers=headers, timeout=5)
        r.encoding = 'utf8'
        soup = BeautifulSoup(r.text, 'html.parser')
        stockName = soup.select_one('td.compName b').text.strip()
        stockName = stockName[stockName.index('公司)')+3:stockName.index('公司提供')].strip()
        table = soup.select_one('table.hasBorder')
        trs = table.select('tr')
        th_array = []
        td_array = []
        for idx, t in enumerate(trs):
            # print(idx,':',t.th,',',t.td)
            if t.th == None:
                if idx > 1 and idx < 10:
                    tds = t.find_all('td')
                    if tds[0].text.strip() == '增減百分比' and idx==5:
                        th_array.append('單月年增率')
                        td_array.append(tds[1].text.strip())
                    elif tds[0].text.strip() == '增減百分比' and idx==9:
                        th_array.append('累積年增率')
                        td_array.append(tds[1].text.strip())
                    else:
                        th_array.append(tds[0].text.strip())
                        td_array.append(tds[1].text.strip())
            else:
                if idx != 0:
                    if t.th.text.strip() == '增減百分比' and idx==4:
                        th_array.append('單月年增率')
                        td_array.append(t.td.text.strip())
                    elif t.th.text.strip() == '增減百分比' and idx==8:
                        th_array.append('累積年增率')
                        td_array.append(t.td.text.strip())
                    else:
                        th_array.append(t.th.text.strip())
                        td_array.append(t.td.text.strip())
        
        if (month-1) == 0:
            payload = getPayload(stockCode, year-1, 12)
        else:
            payload = getPayload(stockCode, year, month-1)
        
        time.sleep(2)
        month = '{:02d}'.format(month)
        r = requests.post('https://mops.twse.com.tw/mops/web/t05st10_ifrs', data=payload, headers=headers, timeout=5)
        r.encoding = 'utf8'
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.select_one('table.hasBorder')
        if table == None:
            th_array.append('上月營收')
            td_array.append(None)
        else:
            trs = table.select('tr')
            for t in trs:
                if t.th == None:
                    tds = t.find_all('td')
                    if tds[0].text.strip() == '本月':
                        th_array.append('上月營收')
                        td_array.append(tds[1].text.strip())
                else:
                    if t.th.text.strip() == '本月':
                        th_array.append('上月營收')
                        td_array.append(t.td.text.strip())
 
    except Exception as e:
        year += 1911
        print('stockCode:'+stockCode+', '+str(year)+str(month)+' 抓資料有問題：'+str(e))
    finally:
        s.close()
        print('Session closed!')
    
    if td_array == None:
        return
    df = pd.DataFrame(np.array([td_array]),columns=th_array)
    year = year+1911
    return df
 
def sendMsg(msg):
  URL = 'https://notify-api.line.me/api/notify'
  headers = {'Authorization': 'Bearer ' + '2wNtEnwpoBvIPHUqagBT5WT3ItzpDlDMLjSsa8BZzwK',}
  payload = {'message': msg}
 
  r = requests.post(URL, headers = headers, params = payload)
  if r.status_code == 200:
    print('send to LINE ok!')
  else:
    print('sendto LINE error!',r.text)
 
def getOutStockNum(stockNo):
  rlt = 0
  try:
    url = 'https://mops.twse.com.tw/mops/web/ajax_t05st03'
    r = requests.post(url, {
            'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'keyword4': '',
            'code1': '',
            'TYPEK2': '',
            'checkbtn': '',
            'queryName': 'co_id',
            'inpuType': 'co_id',
            'TYPEK': 'all',
            'co_id': stockNo,
        })
    # r.encoding = 'utf8'
    content = r.content.decode()
    soup = BeautifulSoup(content, 'html.parser')
    pubStockNum = soup.find('td', text = re.compile('含私募')).text
    pubStockNum = pubStockNum.replace(',','')
    pubStockNum = pubStockNum[:pubStockNum.index('股')]
    rlt = int(pubStockNum)
  except Exception as e:
    print('getOutStockNum Exception:',e)
  # print(pubStockNum)
  return rlt
 
def getAllStockTWSE():
  allStockNo = []
  allStockName = {}
  allStockType = {}
  url = "https://api.finmindtrade.com/api/v4/data"
  parameter = {
      "dataset": "TaiwanStockInfo",
      "token": "", # 參考登入，獲取金鑰
  }
  resp = requests.get(url, params=parameter)
  data = resp.json()
  data = pd.DataFrame(data["data"])
  data = data.loc[data['type']=='twse']
  try:
    for index, row in data.iterrows():
      allStockNo.append(row['stock_id'])
      allStockName[row['stock_id']] = row['stock_name']
      allStockType[row['stock_id']] = row['industry_category']
  except Exception as e:
    pass
  return allStockNo,allStockName,allStockType

def getAllStockTPEX():
  allStockNo = []
  allStockName = {}
  allStockType = {}
  url = "https://api.finmindtrade.com/api/v4/data"
  parameter = {
      "dataset": "TaiwanStockInfo",
      "token": "", # 參考登入，獲取金鑰
  }
  resp = requests.get(url, params=parameter)
  data = resp.json()
  data = pd.DataFrame(data["data"])
  data = data.loc[data['type']=='tpex']
  try:
    for index, row in data.iterrows():
      allStockNo.append(row['stock_id'])
      allStockName[row['stock_id']] = row['stock_name']
      allStockType[row['stock_id']] = row['industry_category']
  except Exception as e:
    pass
  return allStockNo,allStockName,allStockType
 
def getAllStock():
  allStockNoTwse,allStockNameTwse,allStockTypeTwse = getAllStockTWSE()
  allStockNoTpex,allStockNameTpex,allStockTypeTpex = getAllStockTPEX()
  allStockNo = []
  allStockNo.append(allStockNoTwse)
  allStockNo.append(allStockNoTpex)
 
  allStockName = allStockNameTwse.copy()
  allStockName.update(allStockNameTpex)
 
  allStockType = allStockTypeTwse.copy()
  allStockType.update(allStockTypeTpex)
  return allStockNo,allStockName,allStockType
 
def get_STOCK_DAY(run_date, stockNo):
  time.sleep(5)
  # print('查詢日期：',run_date)
  # print('股票代碼',stockNo)
  try:
    link = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d='+run_date+'&stkno='+stockNo
    print(link)
    resp = requests.get(link)
    resp_data = resp.json()
    df = pd.DataFrame(np.array(resp_data['aaData']), columns=['日期','成交張數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數'])
    df = df.set_index('日期')
    # print(df)
  except Exception as e:
    print('get_STOCK_DAY Exception:',e)
  return df
 
def getShiftNdays(result,nDays):
  try:
    result_shift = result.shift(nDays)
    result_shift.columns = [str(nDays)+'day_成交股數',
                str(nDays)+'day_成交金額',
                str(nDays)+'day_開盤價',
                str(nDays)+'day_最高價',
                str(nDays)+'day_最低價',
                str(nDays)+'day_收盤價',
                str(nDays)+'day_漲跌價差',
                str(nDays)+'day_成交筆數',
                str(nDays)+'day_5ma'
                ]
  except Exception as e:
    print('getShiftNdays Exception:',e)
 
  return result_shift
 
def mergeResult(result, result_shift):
  try:
    result_merge = pd.merge(result, result_shift, left_index=True, right_index=True)
    result_merge = result_merge.dropna()
  except Exception as e:
    print('mergeResult Exception:',e)
  return result_merge
  
def main(allStockNo,compareType,run_case,nDays,num_ma,proportion,monthly_report_all_df,allStockType):
  run_date = datetime.datetime.now()
  rlt1 = []
  rlt2 = []
  
  # for stockNo in allStockNo:
  for i, stockNo in enumerate(allStockNo):
    if 'ETF' in allStockType[stockNo] or 'ETN' in allStockType[stockNo]:
      continue

    if i%10 == 0:
      output.clear()
      
    run_case_rlt = {1:True, 2:True, 3:True, 4:True}
    try:
      result = get_STOCK_DAY(run_date.strftime('%Y%m%d'),str(stockNo))
      if run_date < datetime.datetime(int(run_date.strftime('%Y')), int(run_date.strftime('%m')), 10): 
        df_before_month = get_STOCK_DAY((run_date-relativedelta(months=1)).strftime('%Y%m%d'),stockNo)
        result = pd.concat([df_before_month,result],axis=0)
      # print(result)
      result = result.replace('--', np.nan)
      result = result.dropna()
      
      five_ma = result[compareType].rolling(num_ma).mean()
      # print(five_ma)
      result['5ma'] = five_ma
      
      result_shift = getShiftNdays(result,nDays)
      result = mergeResult(result, result_shift)
      result = result.dropna()
      # print(result)

      row = result.tail(1)
      for case in run_case:
        if case == 1: #今天收盤>=5MA
          if (float(row[compareType].values[0].replace(',','')) >= float(row['5ma'].values[0])):
            run_case_rlt[case] = True
          else:
            run_case_rlt[case] = False
        
        if case == 2: #nDays天前的收盤價<5ma
          if (float(row[str(nDays)+'day_'+compareType].values[0].replace(',','')) < float(row[str(nDays)+'day_5ma'].values[0])):
            run_case_rlt[case] = True
          else:
            run_case_rlt[case] = False
        
        if case == 3: #成交比例>0.01
          outStockNum = getOutStockNum(stockNo)
          if outStockNum != 0:
            complete = row['成交張數'].values[0].replace(',','')
            if (float(complete)*1000/outStockNum) >= proportion:
              run_case_rlt[case] = True
            else:
              run_case_rlt[case] = False
          else:
            run_case_rlt[case] = False
        
        if case == 4: #月營收年增率>0:
          if float(monthly_report_all_df.loc[str(stockNo)]['單月年增率']) > 0:
            run_case_rlt[case] = True
          else:
            run_case_rlt[case] = False

      # #今天收盤>=5MA & nDays天前的收盤價<5ma
      # if (float(row[compareType].values[0].replace(',','')) >= float(row['5ma'].values[0])) and \
      # (float(row[str(nDays)+'day_'+compareType].values[0].replace(',','')) < float(row[str(nDays)+'day_5ma'].values[0])): 
      #   outStockNum = getOutStockNum(stockNo)
      #   if outStockNum != 0:
      #     complete = row['成交張數'].values[0].replace(',','')
      #     if (float(complete)*1000/outStockNum) >= proportion: #成交比例 > 0.01
      #       isTrue.append(stockNo)
      #       print(isTrue)

      if run_case_rlt[1] and run_case_rlt[3] and run_case_rlt[4]:
        rlt1.append(stockNo)
        print('rlt1:',rlt1)
    
      if run_case_rlt[1] and run_case_rlt[2] and run_case_rlt[3] and run_case_rlt[4]:
        rlt2.append(stockNo)
        print('rlt2:',rlt2)

        # check1 = True
        # for key, case_rlt in run_case_rlt.items():
        #   check1 = check1 and case_rlt
        
        # if check1:
        #   rlt1.append(stockNo)
        #   print(rlt1)
    
    except Exception as e:
      print('run Exception:',e)
  return rlt1,rlt2
 


In [ ]:
if __name__ == "__main__":
  allStockNo,allStockName,allStockType = getAllStockTPEX()

  #取得當月營收
  monthly_report_date = datetime.datetime.now()-relativedelta(months=1)
  monthly_report_all_df = monthly_report_all(monthly_report_date.year,monthly_report_date.month)
  monthly_report_all_df= monthly_report_all_df.set_index('公司代號')

  case_dict = {1:'一、今日收盤 >= 5MA \n',
          2:'二、前1天的收盤價 < 前1天的5ma \n',
          3:'三、成交比例 > 0.01 \n',
          4:'四、'+monthly_report_date.strftime('%Y%m')+'月營收年增率 > 0\n'}

  nDays = 1 #比對n天前
  num_ma = 5 #計算幾ma
  proportion = 0.01 #比較成交比例
  compareType = '收盤價' #"開盤價","最高價","最低價","收盤價"
  run_case = [1, 2, 3, 4]

  #執行主程式
  rlt1,rlt2 = main(allStockNo,compareType,run_case,nDays,num_ma,proportion,monthly_report_all_df,allStockType)

  print('rlt1:',rlt1)
  run_case = [1, 3, 4]
  title = '上櫃公司執行條件：\n'
  for case in run_case:
    title = title + case_dict[case]

  msg = title
  for i,stockno in enumerate(rlt1):
    msg = msg + str(stockno) + ' \n' \
          # + allStockName[stockno] + ' ' \
          # + allStockType[stockno] + ' '\
          # + '營收年增率：' + str(round(monthly_report_all_df.loc[stockno]['營業收入-去年同月增減(%)'],2)) + '%\n'
    
    # if (i+1)%20 == 0:
    #   if (i+1)==len(rlt1):
    #     msg = msg + '共' + str(len(rlt1)) + '檔股票'
    #   print('msg:',msg)
    #   sendMsg(msg)
    #   msg = title
  
  msg = msg + '共' + str(len(rlt1)) + '檔股票'
  print('msg:',msg)
  sendMsg(msg)

  print('rlt2:',rlt2)
  run_case = [1, 2, 3, 4]
  title = '上櫃公司執行條件：\n'
  for case in run_case:
    title = title + case_dict[case]

  msg = title
  for i,stockno in enumerate(rlt2):
    msg = msg + str(stockno) + ' \n' \
          # + allStockName[stockno] + ' ' \
          # + allStockType[stockno] + ' '\
          # + '營收年增率：' + str(round(monthly_report_all_df.loc[stockno]['營業收入-去年同月增減(%)'],2)) + '%\n'
    
    # if (i+1)%20 == 0:
    #   if (i+1)==len(rlt2):
    #     msg = msg + '共' + str(len(rlt2)) + '檔股票'
    #   print('msg:',msg)
    #   sendMsg(msg)
    #   msg = title

  msg = msg + '共' + str(len(rlt2)) + '檔股票'
  print('msg:',msg)
  sendMsg(msg)